# Generating object masks from input prompts with SAM

In [1]:
# %pip install segment-geospatial

Note: you may need to restart the kernel to use updated packages.


In [2]:
# %pip install xarray

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.1/1.1 MB 2.4 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.1 MB 3.4 MB/s eta 0:00:01
   ------------------ --------------------- 0.5/1.1 MB 4.2 MB/s eta 0:00:01
   ---------------------------- ----------- 0.8/1.1 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# %pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.2.2%2Bcu121-cp311-cp311-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torch-2.2.2%2Bcu121-cp311-cp311-win_amd64.whl (2454.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.2.0
    Uninstalling torch-2.2.0:
      Successfully uninstalled torch-2.2.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import leafmap
from samgeo import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff, split_raster, merge_rasters, raster_to_vector
from samgeo.text_sam import LangSAM
import xarray
import torch

# Plotting the Satellite imagery

In [3]:
image = "C:/Users/cscar/Documents/EMC/Projects/Sunnybrooke/Data/Rectified/Tifs/Sunnybrook_2018.tif"

In [4]:
m = leafmap.Map(center=[43.722069426835596, -79.37448883804542], zoom=18, height="1000px")
m.add_basemap("SATELLITE")
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m 

Map(center=[43.722069426835596, -79.37448883804542], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [5]:
# if m.user_roi_bounds() is not None:
#     bbox = m.user_roi_bounds()
# else:
#     bbox = [-122.2659, 37.8682, -122.2521, 37.8741]

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

In [5]:
# Define the input and output directories
tiles_dir = "C:/Users/cscar/Sunnybrook/data/2021/Tiles/"  # Directory containing the image tiles
masks_dir = "C:/Users/cscar/Sunnybrook/data/2021/Masks/"   # Directory to save the masks
results_dir = "C:/Users/cscar/Sunnybrook/Results/2021/"  # Directory to save the results

In [6]:
checkpoint = os.path.join(results_dir, 'sam_vit_h_4b8939.pth')

In [7]:
sam = SamGeo(
    model_type="vit_h",
    automatic = False,
    sam_kwargs=None,
    checkpoint=checkpoint,
    device = "cuda"
)

In [8]:
sam.set_image(image)

In [9]:
m = sam.show_map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [31]:
split_raster(image, out_dir=out_dir, tile_size=(1000, 1000), overlap=0)

In [10]:
# List all image tiles
tile_files = [f for f in os.listdir(tiles_dir) if os.path.isfile(os.path.join(tiles_dir, f))]

In [13]:
# Process each tile
for tile_file in tile_files:
    tile_path = os.path.join(tiles_dir, tile_file)
    output_path = os.path.join(masks_dir, f"mask_{tile_file}")
    
    # Assuming the method to generate masks is called 'generate'
    # You will need to adjust this according to the actual method's parameters and functionality
    sam.generate(source=tile_path, output=output_path)
    
    print(f"Processed {tile_path} and saved mask to {output_path}")

Processed C:/Users/cscar/Sunnybrook/Results/tile_0_0.tif and saved mask to C:/Users/cscar/Sunnybrook/Masks/mask_tile_0_0.tif
Processed C:/Users/cscar/Sunnybrook/Results/tile_0_1.tif and saved mask to C:/Users/cscar/Sunnybrook/Masks/mask_tile_0_1.tif
Processed C:/Users/cscar/Sunnybrook/Results/tile_0_2.tif and saved mask to C:/Users/cscar/Sunnybrook/Masks/mask_tile_0_2.tif
Processed C:/Users/cscar/Sunnybrook/Results/tile_0_3.tif and saved mask to C:/Users/cscar/Sunnybrook/Masks/mask_tile_0_3.tif
Processed C:/Users/cscar/Sunnybrook/Results/tile_0_4.tif and saved mask to C:/Users/cscar/Sunnybrook/Masks/mask_tile_0_4.tif
Processed C:/Users/cscar/Sunnybrook/Results/tile_0_5.tif and saved mask to C:/Users/cscar/Sunnybrook/Masks/mask_tile_0_5.tif
Processed C:/Users/cscar/Sunnybrook/Results/tile_0_6.tif and saved mask to C:/Users/cscar/Sunnybrook/Masks/mask_tile_0_6.tif
Processed C:/Users/cscar/Sunnybrook/Results/tile_0_7.tif and saved mask to C:/Users/cscar/Sunnybrook/Masks/mask_tile_0_7.tif


In [26]:
output = "C:/Users/cscar/Sunnybrook/Data/Sunnybrook_2018_mask.tif"


In [31]:
merge_rasters(masks_dir, output, input_pattern='*.tif', output_format='GTiff', output_nodata=None, output_options=['COMPRESS=DEFLATE'])

In [32]:
sam.raster_to_vector(image = output, output= output_vector)

In [35]:
sam = LangSAM()

GroundingDINO_SwinB.cfg.py:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

groundingdino_swinb_cogcoor.pth:   0%|          | 0.00/938M [00:00<?, ?B/s]

In [38]:
text_prompt = "tree"

In [36]:
tiles_dir = "C:/Users/cscar/Sunnybrook/Results/Tiles/"
out_dir = "C:/Users/cscar/Sunnybrook/Results/Trees"  # Directory to save the results

In [46]:
sam.predict_batch(
    images= tiles_dir,
    out_dir= out_dir,
    text_prompt=text_prompt,
    box_threshold=0.6,
    text_threshold=0.6,
    mask_multiplier=255,
    dtype="uint8",
    merge=True,
    verbose=True,
)

Processing image 001 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\tile_0_0.tif...
No objects found in the image.
Processing image 002 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\tile_0_1.tif...
No objects found in the image.
Processing image 003 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\tile_0_2.tif...
No objects found in the image.
Processing image 004 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\tile_0_3.tif...
No objects found in the image.
Processing image 005 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\tile_0_4.tif...
No objects found in the image.
Processing image 006 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\tile_0_5.tif...
No objects found in the image.
Processing image 007 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\tile_0_6.tif...
No objects found in the image.
Processing image 008 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\tile_0_7.tif...
No objects found in the image.
Processing image 009 of 112: C:/Users/cscar/Sunnybrook/Results/Tiles\til

In [42]:
output_mask = "C:/Users/cscar/Sunnybrook/Results/Tree_mask.tif"
output_vector = "C:/Users/cscar/Sunnybrook/Results/Sunnybrook_2018_mask.shp"

In [43]:
merge_rasters(out_dir, output_mask, input_pattern='*.tif', output_format='GTiff', output_nodata=None, output_options=['COMPRESS=DEFLATE'])

In [44]:
sam.raster_to_vector(image = output_mask, output= output_vector)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

In [ ]:
leafmap.image_comparison(
    "satellite.tif",
    "annotations.tif",
    label1="Satellite Image",
    label2="Image Segmentation",
)

In [ ]:
m.add_raster("annotations.tif", alpha=0.5, layer_name="Masks")
m